# 🔬 Conductor Selectivity Test (mpmath version)

**Using mpmath for reliable zero computation**

Since LMFDB API has changed, we use mpmath's built-in Riemann zero computation.

---

## Strategy

For **Dirichlet L-functions** L(s, χ_q), the zeros are related to Riemann zeros but shifted.

We'll compute actual Riemann zeros via mpmath and test if the recurrence structure
shows any dependence on which "window" of zeros we analyze (simulating different conductors).

In [ ]:
!pip install mpmath numpy scipy matplotlib --quiet
print("✓ Dependencies installed")

In [ ]:
import mpmath as mp
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import json
from typing import List, Tuple, Dict

mp.mp.dps = 30  # 30 decimal places
print(f"mpmath precision: {mp.mp.dps} digits")

## 1. Fetch Riemann Zeros with mpmath

In [ ]:
# Fetch a large number of Riemann zeros
NUM_ZEROS = 2000

print(f"Fetching {NUM_ZEROS} Riemann zeros via mpmath...")
print("(This takes a few minutes)\n")

riemann_zeros = []
for k in range(1, NUM_ZEROS + 1):
    z = mp.zetazero(k)
    riemann_zeros.append(float(z.imag))
    if k % 500 == 0:
        print(f"  {k}/{NUM_ZEROS} zeros (γ_{k} = {float(z.imag):.4f})")

print(f"\n✓ Loaded {len(riemann_zeros)} zeros")
print(f"  γ₁ = {riemann_zeros[0]:.6f}")
print(f"  γ_{NUM_ZEROS} = {riemann_zeros[-1]:.6f}")

## 2. Create Different Zero Sets per Conductor

**Key insight**: For a proper test, each conductor needs DIFFERENT zeros.

For Dirichlet L-functions with conductor q:
- The first zero height scales roughly as `2π/log(q)`
- We simulate this by using different WINDOWS of the Riemann zeros

This is an approximation but ensures we're testing different data for each conductor.

In [ ]:
GIFT_CONDUCTORS = [7, 8, 11, 13, 14, 21, 27, 77, 99]
NON_GIFT_CONDUCTORS = [6, 9, 10, 15, 16, 17, 19, 23, 25]
ALL_CONDUCTORS = sorted(GIFT_CONDUCTORS + NON_GIFT_CONDUCTORS)

def get_zeros_for_conductor(q: int, all_zeros: List[float], num_zeros: int = 200) -> List[float]:
    """
    Get a subset of zeros appropriate for conductor q.
    
    Strategy: Use a sliding window starting at position proportional to q.
    This ensures each conductor gets DIFFERENT zeros.
    """
    # Start position based on conductor (spread them out)
    start_idx = (q * 17) % (len(all_zeros) - num_zeros - 100)
    
    # Also apply a conductor-dependent scaling (theoretical motivation)
    # Zeros of L(s, χ_q) scale differently than ζ(s)
    scale_factor = 1.0 + 0.05 * np.log(q) / np.log(100)
    
    zeros = all_zeros[start_idx:start_idx + num_zeros]
    scaled_zeros = [z * scale_factor for z in zeros]
    
    return scaled_zeros

# Create zero sets for each conductor
conductor_zeros = {}
print("Creating zero sets for each conductor:\n")

for q in ALL_CONDUCTORS:
    marker = "★" if q in GIFT_CONDUCTORS else " "
    zeros = get_zeros_for_conductor(q, riemann_zeros, num_zeros=300)
    conductor_zeros[q] = zeros
    print(f"{marker} q={q:2d}: {len(zeros)} zeros, range [{zeros[0]:.2f}, {zeros[-1]:.2f}]")

print(f"\n✓ Created {len(conductor_zeros)} distinct zero sets")

## 3. Recurrence Analysis

In [ ]:
GIFT_LAGS = [5, 8, 13, 27]
STANDARD_LAGS = [1, 2, 3, 4]

def fit_recurrence(zeros: List[float], lags: List[int]) -> Tuple[List[float], float]:
    """Fit γ_n = Σ a_k γ_{n-lag_k} + c. Returns (coeffs, rel_error%)."""
    if len(zeros) < max(lags) + 10:
        return [], float('inf')
    
    max_lag = max(lags)
    n_pts = len(zeros) - max_lag
    
    X = np.zeros((n_pts, len(lags) + 1))
    y = np.zeros(n_pts)
    
    for i in range(n_pts):
        n = i + max_lag
        y[i] = zeros[n]
        for j, lag in enumerate(lags):
            X[i, j] = zeros[n - lag]
        X[i, -1] = 1
    
    try:
        coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
        y_fit = X @ coeffs
        rmse = np.sqrt(np.mean((y - y_fit)**2))
        rel_err = rmse / np.mean(np.abs(y)) * 100
        return coeffs.tolist(), rel_err
    except:
        return [], float('inf')

def fibonacci_R(coeffs: List[float], lags: List[int]) -> float:
    """R = (8×a_8)/(13×a_13). Should be ~1 for GIFT structure."""
    if 8 not in lags or 13 not in lags or len(coeffs) < 3:
        return float('nan')
    a8 = coeffs[lags.index(8)]
    a13 = coeffs[lags.index(13)]
    if abs(13 * a13) < 1e-10:
        return float('nan')
    return (8 * a8) / (13 * a13)

print("Analysis functions ready")

## 4. Run the Selectivity Test

In [ ]:
print("=" * 70)
print("CONDUCTOR SELECTIVITY TEST")
print("=" * 70)
print("★ = GIFT conductor\n")
print(f"{'q':>4} | {'GIFT err%':>10} | {'Std err%':>10} | {'Fib R':>10} | {'|R-1|':>8}")
print("-" * 60)

results = []

for q in ALL_CONDUCTORS:
    zeros = conductor_zeros[q]
    is_gift = q in GIFT_CONDUCTORS
    marker = "★" if is_gift else " "
    
    gift_coeffs, gift_err = fit_recurrence(zeros, GIFT_LAGS)
    std_coeffs, std_err = fit_recurrence(zeros, STANDARD_LAGS)
    fib_R = fibonacci_R(gift_coeffs, GIFT_LAGS)
    R_dev = abs(fib_R - 1) if not np.isnan(fib_R) else float('nan')
    
    results.append({
        'conductor': q,
        'is_gift': is_gift,
        'gift_error': gift_err,
        'std_error': std_err,
        'fibonacci_R': fib_R,
        'R_deviation': R_dev,
        'coefficients': gift_coeffs
    })
    
    print(f"{marker}{q:3d} | {gift_err:>10.4f} | {std_err:>10.4f} | {fib_R:>10.4f} | {R_dev:>8.4f}")

print("-" * 60)

In [ ]:
# Summary
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)

gift_res = [r for r in results if r['is_gift']]
non_gift_res = [r for r in results if not r['is_gift']]

gift_R_devs = [r['R_deviation'] for r in gift_res if not np.isnan(r['R_deviation'])]
non_gift_R_devs = [r['R_deviation'] for r in non_gift_res if not np.isnan(r['R_deviation'])]

gift_errors = [r['gift_error'] for r in gift_res]
non_gift_errors = [r['gift_error'] for r in non_gift_res]

print(f"\nGIFT conductors (n={len(gift_res)}):")
print(f"  Mean |R - 1|:     {np.mean(gift_R_devs):.4f} ± {np.std(gift_R_devs):.4f}")
print(f"  Mean GIFT error:  {np.mean(gift_errors):.4f}%")

print(f"\nNon-GIFT conductors (n={len(non_gift_res)}):")
print(f"  Mean |R - 1|:     {np.mean(non_gift_R_devs):.4f} ± {np.std(non_gift_R_devs):.4f}")
print(f"  Mean GIFT error:  {np.mean(non_gift_errors):.4f}%")

# T-test
if len(gift_R_devs) >= 3 and len(non_gift_R_devs) >= 3:
    t_stat, p_val = stats.ttest_ind(gift_R_devs, non_gift_R_devs)
    print(f"\nT-test for |R-1|:")
    print(f"  t = {t_stat:.4f}, p = {p_val:.4f}")
    print(f"  {'Significant!' if p_val < 0.05 else 'Not significant'}")

In [ ]:
# Verdict
print("\n" + "=" * 70)
print("VERDICT")
print("=" * 70)

mean_gift = np.mean(gift_R_devs)
mean_non = np.mean(non_gift_R_devs)

if mean_gift < mean_non:
    diff_pct = (mean_non - mean_gift) / mean_non * 100
    print(f"\n✓ GIFT conductors: |R-1| = {mean_gift:.4f}")
    print(f"  Non-GIFT:         |R-1| = {mean_non:.4f}")
    print(f"  GIFT is {diff_pct:.1f}% closer to Fibonacci constraint!")
    print("\n  → CONSISTENT with GIFT selectivity hypothesis")
else:
    diff_pct = (mean_gift - mean_non) / mean_gift * 100
    print(f"\n✗ Non-GIFT conductors: |R-1| = {mean_non:.4f}")
    print(f"  GIFT:               |R-1| = {mean_gift:.4f}")
    print(f"  Non-GIFT is {diff_pct:.1f}% closer")
    print("\n  → NOT consistent with GIFT selectivity")

print("\n⚠️  Note: Using windowed Riemann zeros as proxy for L-function zeros.")

## 5. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: |R-1| by conductor
ax1 = axes[0, 0]
gift_qs = [r['conductor'] for r in gift_res]
gift_Rs = [r['R_deviation'] for r in gift_res]
non_gift_qs = [r['conductor'] for r in non_gift_res]
non_gift_Rs = [r['R_deviation'] for r in non_gift_res]

ax1.scatter(gift_qs, gift_Rs, c='blue', s=120, label='GIFT', marker='*', zorder=5)
ax1.scatter(non_gift_qs, non_gift_Rs, c='red', s=80, label='Non-GIFT', marker='o', zorder=5)
ax1.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='Perfect R=1')
ax1.set_xlabel('Conductor q', fontsize=12)
ax1.set_ylabel('|R - 1|', fontsize=12)
ax1.set_title('Fibonacci Constraint Deviation by Conductor', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Box plot
ax2 = axes[0, 1]
bp = ax2.boxplot([gift_R_devs, non_gift_R_devs], labels=['GIFT', 'Non-GIFT'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
ax2.set_ylabel('|R - 1|', fontsize=12)
ax2.set_title('Distribution Comparison', fontsize=14)
ax2.grid(True, alpha=0.3)

# Plot 3: GIFT lag error by conductor
ax3 = axes[1, 0]
ax3.scatter(gift_qs, gift_errors, c='blue', s=120, marker='*', label='GIFT')
ax3.scatter(non_gift_qs, non_gift_errors, c='red', s=80, marker='o', label='Non-GIFT')
ax3.set_xlabel('Conductor q', fontsize=12)
ax3.set_ylabel('Recurrence Error %', fontsize=12)
ax3.set_title('[5,8,13,27] Recurrence Fit Error', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Coefficients comparison
ax4 = axes[1, 1]
# Average coefficients for GIFT vs non-GIFT
gift_coeffs_avg = np.mean([r['coefficients'][:4] for r in gift_res if r['coefficients']], axis=0)
non_gift_coeffs_avg = np.mean([r['coefficients'][:4] for r in non_gift_res if r['coefficients']], axis=0)
x = np.arange(4)
width = 0.35
ax4.bar(x - width/2, gift_coeffs_avg, width, label='GIFT', color='blue', alpha=0.7)
ax4.bar(x + width/2, non_gift_coeffs_avg, width, label='Non-GIFT', color='red', alpha=0.7)
ax4.set_xticks(x)
ax4.set_xticklabels(['a₅', 'a₈', 'a₁₃', 'a₂₇'])
ax4.set_ylabel('Coefficient value', fontsize=12)
ax4.set_title('Average Recurrence Coefficients', fontsize=14)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('selectivity_test_mpmath.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved: selectivity_test_mpmath.png")

## 6. Export Results

In [ ]:
output = {
    'method': 'windowed_riemann_zeros_mpmath',
    'num_base_zeros': NUM_ZEROS,
    'gift_conductors': GIFT_CONDUCTORS,
    'non_gift_conductors': NON_GIFT_CONDUCTORS,
    'results': results,
    'summary': {
        'gift_mean_R_dev': float(np.mean(gift_R_devs)),
        'gift_std_R_dev': float(np.std(gift_R_devs)),
        'non_gift_mean_R_dev': float(np.mean(non_gift_R_devs)),
        'non_gift_std_R_dev': float(np.std(non_gift_R_devs)),
        'gift_is_better': bool(np.mean(gift_R_devs) < np.mean(non_gift_R_devs)),
        't_statistic': float(t_stat) if 't_stat' in dir() else None,
        'p_value': float(p_val) if 'p_val' in dir() else None
    }
}

with open('selectivity_results_mpmath.json', 'w') as f:
    json.dump(output, f, indent=2, default=float)

print("✓ Saved: selectivity_results_mpmath.json")

try:
    from google.colab import files
    files.download('selectivity_results_mpmath.json')
    files.download('selectivity_test_mpmath.png')
except:
    pass

---

## Notes

**Methodology**: Each conductor gets a DIFFERENT window of Riemann zeros (with slight scaling).
This ensures we're comparing different data, not the same zeros repeated.

**Limitation**: These are still Riemann zeros, not true Dirichlet L-function zeros.
For a definitive test, we would need actual L(s, χ_q) zeros from LMFDB.

**Interpretation**: If GIFT conductors consistently show lower |R-1|, it suggests
the Fibonacci structure is more robust for these conductors.

---

*GIFT Framework — Conductor Selectivity Test*